In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the fhuA gene
model0['lb_list']['EX_fecrm_e ']=0
model0['lb_list']['EX_fecrm_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./polymicroorganism/km.csv')
vmax = pd.read_csv('./polymicroorganism/vmax.csv')
public_metabolism = pd.read_csv('./polymicroorganism/public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_s, micro_distribute_threshold, length, D):
    
    number_cell_side = length // deta_s
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro})
    print(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
    
    # iterative simulation by slip_r
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = int(micro_decrease_cell)
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = int(micro_increase_fcell)
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = int(micro_decrease_cell)
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = int(micro_increase_fcell)
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = int(micro_decrease_cell)
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = int(micro_increase_bcell)
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = int(micro_decrease_cell)
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = int(micro_increase_bcell)
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = int(micro_decrease_cell)
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = int(micro_increase_fcell)
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_s-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_s)/deta_s*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_s/deta_s*D*deta_t
                distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_s/deta_s*D*deta_t
                distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        for m in range(number_cell_side):
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
        for m in range(number_cell_side):
            B_value_list = []
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = int(public_reaction_ub_list[i][j][0,m])
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    try:
                        MDF_Calculation(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
                    except:
                        save_rate = 0.9
                        while save_rate >= 0:
                            biomass_value2 = save_rate * biomass_value
                            try:
                                MDF_Calculation(model_list[i],biomass_value2,biomass_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
                            except:
                                save_rate = save_rate - 0.1
                            else:
                                #calculate the MDF values of metabolic networks
                                biomass_value_micro[i] = biomass_value2
                                B_value2 = MDF_Calculation(model_list[i],biomass_value2,biomass_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
                                #calculate the biomass yield under the MDF value
                                obj_name=biomass_list[i]
                                obj_target='maximize'
                                if i == 0:
                                    max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value2,'gurobi')
                                elif i == 1:
                                    max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value2,'gurobi')
                                biomass_value=max_biomass_under_mdf*0.9
                                
                                #calculate the minimum value of the sum of the fluxes
                                if i == 0:
                                    [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                                elif i == 1:
                                    [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                                
                                #translating the results of ETMs into a usable form
                                model=model_list[i]['model']
                                reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                                reaction_g0=model_list[i]['reaction_g0']
                                coef_matrix=model_list[i]['coef_matrix']
                                metabolite_list=model_list[i]['metabolite_list']
                                use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
                                distribute_public_metabolism_increase = {}
                                distribute_public_metabolism_decrease = {}
                                
                                #simulate the fluxes of the public metabolites
                                for rea in public_reaction_list[i]:
                                    for met in public_metabolism_name_list:
                                        try:
                                            model_list[i]['coef_matrix'][(met,rea)]
                                        except:
                                            pass
                                        else:
                                            public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                                
                                #simulate the distribution of the public metabolites
                                for met in public_metabolism_name_list:
                                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]*save_rate
                                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
                                    
                                    
                                save_rate_ub = [save_rate]
                                for rea in public_reaction_list[i]:
                                    if 'reverse' not in rea:
                                        met = rea[3::]
                                        if met in public_metabolism_name_list:
                                    
                                            intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                            intracellular_c = intracellular_c.replace(';',',"')
                                            intracellular_c = intracellular_c.replace(' :','" :')
                                            intracellular_c = '{"' + intracellular_c + '}'
                                            intracellular_c_dict = ast.literal_eval(intracellular_c)
                                        
                                            save_rate_met = distribute_public_metabolism_list[met][0,m]/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                            save_rate_ub.append(save_rate_met)
                                        
                                save_rate_final = min(save_rate_ub)
                                
                                for rea in public_reaction_list[i]:
                                    if 'reverse' not in rea:
                                        if met in public_metabolism_name_list:
                                            met = rea[3::]
                                    
                                            intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                            intracellular_c = intracellular_c.replace(';',',"')
                                            intracellular_c = intracellular_c.replace(' :','" :')
                                            intracellular_c = '{"' + intracellular_c + '}'
                                            intracellular_c_dict = ast.literal_eval(intracellular_c)
                                        
                                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - intracellular_c_dict[met]*int((save_rate_final)*distribute_micro_list_t[ct][i][m]) + intracellular_c_dict[met]*int((1-save_rate_final)*distribute_micro_list_t[ct][i][m])
                                            if distribute_public_metabolism_list[met][0,m] < 0:
                                                print(met + '_3: ', distribute_public_metabolism_list[met][0,m])
                                    
                                
                                distribute_micro_decrease = int((1-save_rate_final) * distribute_micro_list_t[ct][i][m])/breed_list[i]
                                distribute_micro_increase = int(save_rate_final * distribute_micro_list_t[ct][i][m])/breed_list[i]
                                distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] - distribute_micro_decrease + distribute_micro_increase)
                                
                                break
                            
                        continue
                    else:
                        #calculate the MDF values of metabolic networks
                        biomass_value_micro[i] = biomass_value
                        B_value=MDF_Calculation(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
                        B_value_list.append(B_value)
                        #calculate the biomass yield under the MDF value
                        obj_name=biomass_list[i]
                        obj_target='maximize'
                        if i == 0:
                            max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                        elif i == 1:
                            max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                        biomass_value=max_biomass_under_mdf*0.9
                        
                        #calculate the minimum value of the sum of the fluxes
                        if i == 0:
                            [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
                        elif i == 1:
                            [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
            
                        #translating the results of ETMs into a usable form
                        model=model_list[i]['model']
                        reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                        reaction_g0=model_list[i]['reaction_g0']
                        coef_matrix=model_list[i]['coef_matrix']
                        metabolite_list=model_list[i]['metabolite_list']
                        use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
                        
                        #simulate the fluxes of the public metabolites
                        for rea in public_reaction_list[i]:
                            for met in public_metabolism_name_list:
                                try:
                                    model_list[i]['coef_matrix'][(met,rea)]  
                                except:
                                    pass
                                else:
                                    public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
            
                        #simulate the distribution of the public metabolites
                        distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                        distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                            if distribute_public_metabolism_list[met][0,m] < 0:
                                print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                        #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death                
                        death_rate = 0
                        birth_rate = 1
                        for rea in public_reaction_list[i]:
                            if 'reverse' not in rea:
                                met = rea[3::]
                                if met in public_metabolism_name_list:
                            
                                    intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                    intracellular_c = intracellular_c.replace(';',',"')
                                    intracellular_c = intracellular_c.replace(' :','" :')
                                    intracellular_c = '{"' + intracellular_c + '}'
                                    intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                    bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                    if bd_rate_met < 0:
                                        death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                        death_rate = max(death_rate, death_rate_lb)
                                    else:
                                        birth_rate = min(birth_rate, bd_rate_met)
                        death_rate = min(death_rate, 1)
                        birth_rate = min(birth_rate,1)
                        if death_rate > 0:
                            birth_rate = 0
                            for met in public_metabolism_name_list:
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                                distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        
                        for rea in public_reaction_list[i]:
                            if 'reverse' not in rea:
                                met = rea[3::]
                                if met in public_metabolism_name_list:
                            
                                    intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                    intracellular_c = intracellular_c.replace(';',',"')
                                    intracellular_c = intracellular_c.replace(' :','" :')
                                    intracellular_c = '{"' + intracellular_c + '}'
                                    intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                    if distribute_public_metabolism_list[met][0,m] < 0:
                                        print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                        distribute_public_metabolism_list[met][0,m] = 0
                        
                        #simulate the distribution of strains after multiplication or death
                        distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                        distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                        distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe3+ intake
                                        if 'EX_fe3_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
        
        # calculate the mean number and uniformity of distribution of strains at this iteration
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + ((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m]))**2
                    else:
                        r = r + (distribute_micro_list_t[ct][i][m])**2
        
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-5:])
        fd_r_threshold = r_threshold[5:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
    return(distribute_micro_list_t, number, various)

In [6]:
distribute_micro_list_t, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [4, 6], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/30, 0.1, 0.05, 2, 0.2)
print('The change process of microorganism distribution is: ',distribute_micro_list_t)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 1, 11,  5,  7, 10, 14, 19, 13,  6,  4, 13,  5,  6, 15, 18, 18, 12,
        2,  7]), 1: array([ 6,  4,  5,  9,  4, 13, 12, 13, 19, 18,  5,  5,  1,  3, 16, 19,  0,
        2, 10])}
1
strain_number:  0 11.789473684210526
strain_various:  0 6.6773690640647185
strain_number:  1 9.68421052631579
strain_various:  1 7.211486682106886
fd_r_threshold:  [1.05]
slip_r:  1.05
2
strain_number:  0 14.421052631578947
strain_various:  0 5.761174857099203
strain_number:  1 10.842105263157896
strain_various:  1 5.565525065263614
fd_r_threshold:  [1.05, 88.03006818524854]
slip_r:  18.44601363704971
3
strain_number:  0 17.68421052631579
strain_various:  0 5.867417898682855
strain_number:  1 12.157894736842104
strain_various:  1 4.79323160843262
fd_r_threshold:  [1.05, 88.03006818524854, 10.924313670205606]
slip_r:  20.42087637109083
4
strain_number:  0 21.68421052631579
strain_various:  0 5.749623617694671
strain_number:  1 13.68421052631579
strain_various:  1 4.735672370190312
fd_r_threshold: 

glc__D_e_1:  -3.489993517040485
glc__D_e_1:  -3.095674981204316
glc__D_e_1:  -0.041588638994955796
glc__D_e_1:  -4.628397658462129
glc__D_e_1:  -3.656856911725562
glc__D_e_1:  -1.9256923081697095
glc__D_e_1:  -4.820320770925537
glc__D_e_1:  -4.517454072621481
glc__D_e_1:  -1.1229313259457614
glc__D_e_1:  -0.5292750665995376
glc__D_e_1:  -1.1998324423094222
glc__D_e_1:  -0.7140620689411388
glc__D_e_1:  -2.9489629738954233
glc__D_e_1:  -3.092995492547922
glc__D_e_1:  -3.5357143641288813
glc__D_e_1:  -4.144406900941478
glc__D_e_1:  -1.2427995105471137
glc__D_e_1:  -0.6491432512008899
glc__D_e_1:  -2.7757253086289646
glc__D_e_1:  -3.050080821225306
glc__D_e_1:  -2.0922166904410533
glc__D_e_1:  -3.2198342464251826
glc__D_e_1:  -3.5154413565715252
glc__D_e_1:  -3.455459844951611
glc__D_e_1:  -1.5309814762014766
glc__D_e_1:  -2.343467793056002
glc__D_e_1:  -2.5263714681202294
glc__D_e_1:  -1.5617300055455625
glc__D_e_1:  -1.4598662648651537
glc__D_e_1:  -4.466956332466422
glc__D_e_1:  -3.4954

glc__D_e_1:  -0.8647434356190762
glc__D_e_1:  -0.024008210027022914
glc__D_e_1:  -0.08857963732001495
glc__D_e_1:  -1.578130880355404
glc__D_e_1:  -0.36019940849298393
glc__D_e_1:  -0.45165124602509765
glc__D_e_1:  -0.06700500843055668
glc__D_e_1:  -0.25478165258996865
glc__D_e_1:  -0.6805705143383376
glc__D_e_1:  -0.7808275085676246
glc__D_e_1:  -0.8275089232869159
glc__D_e_1:  -0.6620778883702745
glc__D_e_1:  -0.770411743942691
glc__D_e_1:  -0.050475319346117375
glc__D_e_1:  -0.12699372795231723
glc__D_e_1:  -0.3395798542958066
glc__D_e_1:  -1.0449120384079702
glc__D_e_1:  -1.8050379243725945
glc__D_e_1:  -0.0349282394846433
glc__D_e_1:  -0.33389707398131296
strain_number:  0 4.421052631578948
strain_various:  0 1.4979209969218639
strain_number:  1 3.3157894736842106
strain_various:  1 0.653140718210045
fd_r_threshold:  [1.05, 88.03006818524854, 10.924313670205606, 8.85068780485541, 8.484989143073772, 6.909870423021428, 4.571276838160089, 2.991331590691295, 4.970635988124667, 4.33299

glc__D_e_1:  -0.4256067584617853
glc__D_e_1:  -0.46867042968926775
glc__D_e_1:  -0.3213454161306112
glc__D_e_1:  -0.05963223393242145
glc__D_e_1:  -0.11991922243281228
glc__D_e_1:  -0.6003027973555228
strain_number:  0 1.894736842105263
strain_various:  0 0.7877173445839877
strain_number:  1 0.7894736842105263
strain_various:  1 0.6942582083301537
fd_r_threshold:  [1.05, 88.03006818524854, 10.924313670205606, 8.85068780485541, 8.484989143073772, 6.909870423021428, 4.571276838160089, 2.991331590691295, 4.970635988124667, 4.3329961470543825, 2.4732000865380614, 3.2707051771317124, 2.6952420566900885, 6.3419164099508265, 2.7828043462553964, 2.3824576756016156, 3.1263016502897454, 4.520550516502897, 2.2626318027210885, 2.2392970521541953, 3.1997222222222224, 2.9447222222222225, 4.269444444444444, 3.555555555555556, 1.1736111111111112, 3.8333333333333335]
slip_r:  3.1553333333333335
27
glc__D_e_1:  -0.05317959804940625
glc__D_e_1:  -0.10331561347055496
glc__D_e_1:  -0.14275120793149088
glc_

glc__D_e_1:  -0.1357859166799128
glc__D_e_1:  -0.022212688420815585
strain_number:  0 0.42105263157894735
strain_various:  0 0.5907880084379907
strain_number:  1 0.10526315789473684
strain_various:  1 0.30689220499185793
fd_r_threshold:  [1.05, 88.03006818524854, 10.924313670205606, 8.85068780485541, 8.484989143073772, 6.909870423021428, 4.571276838160089, 2.991331590691295, 4.970635988124667, 4.3329961470543825, 2.4732000865380614, 3.2707051771317124, 2.6952420566900885, 6.3419164099508265, 2.7828043462553964, 2.3824576756016156, 3.1263016502897454, 4.520550516502897, 2.2626318027210885, 2.2392970521541953, 3.1997222222222224, 2.9447222222222225, 4.269444444444444, 3.555555555555556, 1.1736111111111112, 3.8333333333333335, 4.861111111111111, 1.5, 1.8611111111111112, 1.3611111111111112, 3.25, 2.25, 0.25, 2.25, 5.25, 2.0]
slip_r:  2.4
37
strain_number:  0 0.42105263157894735
strain_various:  0 0.5907880084379907
strain_number:  1 0.10526315789473684
strain_various:  1 0.3068922049918579

strain_number:  0 0.2631578947368421
strain_various:  0 0.44034738238635557
strain_number:  1 0.05263157894736842
strain_various:  1 0.22329687826943606
fd_r_threshold:  [1.05, 88.03006818524854, 10.924313670205606, 8.85068780485541, 8.484989143073772, 6.909870423021428, 4.571276838160089, 2.991331590691295, 4.970635988124667, 4.3329961470543825, 2.4732000865380614, 3.2707051771317124, 2.6952420566900885, 6.3419164099508265, 2.7828043462553964, 2.3824576756016156, 3.1263016502897454, 4.520550516502897, 2.2626318027210885, 2.2392970521541953, 3.1997222222222224, 2.9447222222222225, 4.269444444444444, 3.555555555555556, 1.1736111111111112, 3.8333333333333335, 4.861111111111111, 1.5, 1.8611111111111112, 1.3611111111111112, 3.25, 2.25, 0.25, 2.25, 5.25, 2.0, 0.0, 1.0, 0.0, 1.0, 0.25, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
slip_r:  0.2
48
strain_number:  0 0.21052631578947367
strain_various:  0 0.40768245749551757
strain_number:  1 0.05263157894736842
strain_various:  1 0.22329687826943606
fd_r_thre